# CT Slice Quality Assessment with VLLM
Adapted from the LabelCritic repo. This notebook:
1. Loads 5 random CT slices from a specified folder.
2. Prepares a prompt asking the VLLM to rate image quality (1–5).
3. Sends the prompt and images to the VLLM server.
4. Prints out the model's response.


In [ ]:
import os
import random
from PIL import Image
from io import BytesIO
import base64
import requests
from ErrorDetector import CreateConversation  # assumes LabelCritic repo is in your PYTHONPATH

def encode_image(img_path, solid_overlay=False):
    image = Image.open(img_path).convert("RGB")
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [ ]:
# === Configuration ===
ct_slice_dir = "path/to/your/ct_slices"  # <-- CHANGE THIS
vllm_url = "http://localhost:8000/v1/chat/completions"  # Update if your VLLM uses a different port

prompt_text = "Give score 1–5, how good is the image quality?"
num_slices = 5

In [ ]:
# === Sample 5 random CT slices ===
all_slices = [os.path.join(ct_slice_dir, f) for f in os.listdir(ct_slice_dir) if f.lower().endswith(('.png', '.jpg'))]
sampled_slices = random.sample(all_slices, num_slices)

for s in sampled_slices:
    print(f"Selected: {s}")

In [ ]:
# === Create VLLM conversation ===
conversation = CreateConversation(
    img_file_list=sampled_slices,
    text=prompt_text,
    conver=[],
    prt=True
)

In [ ]:
# === Send prompt to VLLM ===
payload = {
    "model": "gpt-4-vision-preview",  # or whatever your model is called
    "messages": conversation,
    "temperature": 0.2
}

response = requests.post(vllm_url, json=payload)
response.raise_for_status()
print("Model response:")
print(response.json()["choices"][0]["message"]["content"])